In [33]:
import pandas as pd
import recordlinkage
import numpy as np
import timeit
from rapidfuzz import process

In [34]:
agg_debt = pd.read_csv('data/agg_debt_grouped.csv')
agg_debt.head()

C:\Users\david\AppData\Local\Temp\ipykernel_15128\786871525.py:1: DtypeWarning: Columns (2,10,11,13,15,16,20,21,22,25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  agg_debt = pd.read_csv('data/agg_debt_grouped.csv')


,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,to whom due | title,time when the debt became due | month,...,amount | 10th,exchange,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.0,C,8.0,27.0,1783.0,Elizabeth,Lowell,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.0,G,8.0,27.0,1783.0,Joshua,Brackett,Esq,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3.0,C,9.0,2.0,1783.0,Phillips,White,Esq,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.0,B,9.0,2.0,1783.0,William,White,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
agg_debt.loc[agg_debt['org_file'] == 'Pierce_Certs_cleaned_2019.xlsx'].head()

,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,to whom due | title,time when the debt became due | month,...,amount | 10th,exchange,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1
14015,14015,15296.0,NaN,NaN,NaN,NaN,Mason,Abbe,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14016,14016,15298.0,NaN,NaN,NaN,NaN,John,Abbee,Jun,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14017,14017,15299.0,NaN,NaN,NaN,NaN,Richard,Abbet,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14018,14018,15300.0,NaN,NaN,NaN,NaN,Reuben,Abbett,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14019,14019,15301.0,NaN,NaN,NaN,NaN,Richard,Abbett,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Remove 'cs' (congress) and 'f' (foreign officers); these are not state, but specific regiments / types of officers
agg_debt_copy = agg_debt[(agg_debt['state'] != 'cs') & (agg_debt['state'] != 'f') & (agg_debt['state'] != 'de')]

# Split dataframe based on state; makes searching faster
agg_debt_sp = agg_debt_copy.groupby('state')
agg_debts_st = [agg_debt_sp.get_group(x) for x in agg_debt_sp.groups]


for st in agg_debt_sp.groups:
    print(st)

ct
ga
ma
md
nc
nh
nj
ny
pa
ri
va


In [39]:
# Remove matches that have a score of 100 
def remove_exact_matches(matches):
    n_matches = []
    for match in matches:
        if match[1] < 100:
            n_matches.append(match)
    return n_matches

In [42]:
%%timeit
for state in agg_debt_sp.groups:
    agg_debt_st = agg_debt_sp.get_group(state)
    # Create a full name column because process.extract can only compare one string at a time
    agg_debt_st['full name'] = agg_debt_st['to whom due | first name'] + ' ' + agg_debt_st['to whom due | last name']
    # Iterate through each state's dataframe and .extract all the matches for each row 
    agg_debt_st['matches'] = agg_debt_st['full name'].apply(lambda full_name: process.extract(full_name, agg_debt_st['full name'], score_cutoff=90))
    print(state, str(len(agg_debt_st['matches'])))
    # Remove matches that have a score of 100 
    agg_debt_st['matches'] = agg_debt_st['matches'].apply(lambda matches: remove_exact_matches(matches))
    # Remove rows that have no matches 
    agg_debt_st_clean = agg_debt_st.loc[agg_debt_st.matches.str.len() > 0]
    # Save as a .csv file
    agg_debt_st_clean.to_csv('data/similar names/similar_names_' + state + '.csv') 

<magic-timeit>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

ct 11335
ga 82


KeyboardInterrupt: 

In [13]:
agg_debt.loc[[90807]]

,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,to whom due | title,time when the debt became due | month,...,amount | 10th,exchange,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1
90807,90807,187350.0,NaN,NaN,NaN,NaN,Parker,Noah,Jun,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
indexer = recordlinkage.Index()
indexer.full()

<Index>

In [6]:
compare = recordlinkage.Compare()
compare.exact('to whom due | first name', 'to whom due | first name', label='first name')
compare.string('to whom due | last name',
            'to whom due | last name',
            method='levenshtein',
            label='last name')

<Compare>

Connecticut

In [5]:
ct_comparisons = indexer.index(agg_debt_sp.get_group('ct'), agg_debt_sp.get_group('ct'))
print(len(ct_comparisons))

128482225


In [16]:
test = features.loc[(features['first name'] == 1) & (features['last name'] >= 0.9) & (features['last name'] < 1)]

In [32]:
print(type(test))

<class 'pandas.core.frame.DataFrame'>


In [31]:
len(test)

60

In [29]:
test

first name  last name
16817  16986            1   0.900000
16986  16817            1   0.900000
26348  102620           1   0.909091
       102658           1   0.909091
       102677           1   0.909091
27650  123289           1   0.900000
       123880           1   0.900000
       123897           1   0.900000
29001  29837            1   0.900000
29837  29001            1   0.900000
31259  31260            1   0.900000
31260  31259            1   0.900000
37762  124080           1   0.900000
37763  37800            1   0.900000
37800  37763            1   0.900000
49978  49986            1   0.900000
49986  49978            1   0.900000
102620 26348            1   0.909091
       102921           1   0.909091
       120962           1   0.909091
       121652           1   0.909091
102658 26348            1   0.909091
       102921           1   0.909091
       120962           1   0.909091
       121652           1   0.909091
102677 26348            1   0.909091
       102921           1   0.909091
       120962           1   0.909091
       121652           1   0.909091
102840 119823           1   0.900000
       119897           1   0.900000
       119900           1   0.900000
       119931           1   0.900000
       119939           1   0.900000
       120191           1   0.900000
       121544           1   0.900000
       121547           1   0.900000
102921 102620           1   0.909091
       102658           1   0.909091
       102677           1   0.909091
119823 102840           1   0.900000
119897 102840           1   0.900000
119900 102840           1   0.900000
119931 102840           1   0.900000
119939 102840           1   0.900000
120173 122784           1   0.900000
120191 102840           1   0.900000
120962 102620           1   0.909091
       102658           1   0.909091
       102677           1   0.909091
121544 102840           1   0.900000
121547 102840           1   0.900000
121652 102620           1   0.909091
       102658           1   0.909091
       102677           1   0.909091
122784 120173           1   0.900000
123289 27650            1   0.900000
123880 27650            1   0.900000
123897 27650            1   0.900000
124080 37762            1   0.900000

In [30]:
agg_debt.iloc[[121652]][['to whom due | first name', 'to whom due | last name', 'state']]

,to whom due | first name,to whom due | last name,state
121652,Ebenezer,Frothingham,ct


In [27]:
agg_debt.iloc[[102620]][['to whom due | first name', 'to whom due | last name', 'state']]

,to whom due | first name,to whom due | last name,state
102620,Ebenezer,Trothingham,ct


In [ ]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'first name':'last name'].sum(axis=1)

In [ ]:
potential_matches.head()

In [18]:
len(potential_matches.loc[potential_matches['level_0'] != potential_matches['level_1']])

18276

In [6]:
from joblib import Parallel, delayed, cpu_count

In [7]:
def compare_strings(st):
    st_comparisons = indexer.index(agg_debt_sp.get_group(st), agg_debt_sp.get_group(st))
    print(st + ': ' + str(len(st_comparisons)))
    features = compare.compute(st_comparisons, agg_debt_sp.get_group(st), agg_debt_sp.get_group(st))
    parse_features = features.loc[(features['first name'] == 1) & (features['last name'] >= 0.9) & (features['last name'] < 1)]
    print(st + ': ' + str(len(parse_features)))
    parse_features.to_csv('data/similar_names_' + st + '.csv') 

In [ ]:
%%time
for st in agg_debt_sp.groups:
    if st != 'ct' and st != 'ga' and st != 'md' and st != 'nc' and st != 'nh' and st != 'ri':
        compare_strings(st)

ma: 776291044


In [7]:
from rapidfuzz import process
agg_debt_ma = agg_debt_sp.get_group('ma')
agg_debt_ma['full name'] = agg_debt_ma['to whom due | first name'] + ' ' + agg_debt_ma['to whom due | last name'] 
agg_debt_ma['full name']

C:\Users\david\AppData\Local\Temp\ipykernel_32340\1082223012.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_debt_ma['full name'] = agg_debt_ma['to whom due | first name'] + ' ' + agg_debt_ma['to whom due | last name']


11443          Isaac Peirce
11444       Benjamin Burley
11445         Ebenezer Clap
11446       Isaac Greenwood
11447           Abner Bacon
                ...        
119754     William Phillips
119755      Ebenezer Storer
119756      Samuel Dashwood
119757           John Tudor
119758    Jonathan Williams
Name: full name, Length: 27862, dtype: object

In [21]:
%%time
# apply the process.extract function on the col1 column
agg_debt_ma['matches'] = agg_debt_ma['full name'].apply(lambda full_name: process.extract(full_name, agg_debt_ma['full name'], score_cutoff=90))

CPU times: total: 2min 38s
Wall time: 12min 3s


<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
agg_debt_ma_copy = agg_debt_ma.copy()

In [23]:
agg_debt_ma.head()

,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,to whom due | title,time when the debt became due | month,...,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1,full name,matches
11443,11443,12593.0,A,8.0,8.0,1783.0,Isaac,Peirce,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Isaac Peirce,"[(Isaac Peirce, 100.0, 11443), (Isaac Peirce, ..."
11444,11444,12594.0,B,8.0,8.0,1783.0,Benjamin,Burley,NaN,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Benjamin Burley,"[(Benjamin Burley, 100.0, 11444), (Benjamin Bu..."
11445,11445,12595.0,C,8.0,9.0,1783.0,Ebenezer,Clap,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ebenezer Clap,"[(Ebenezer Clap, 100.0, 11445), (Ebenezer Clap..."
11446,11446,12596.0,D,8.0,11.0,1783.0,Isaac,Greenwood,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Isaac Greenwood,"[(Isaac Greenwood, 100.0, 11446), (Isaac Green..."
11447,11447,12597.0,E,8.0,11.0,1783.0,Abner,Bacon,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abner Bacon,"[(Abner Bacon, 100.0, 11447), (Abner Bacon, 10..."


In [38]:
agg_debt_ma['matches'] = agg_debt_ma['matches'].apply(lambda matches: remove_exact_matches(matches))

In [39]:
agg_debt_ma['matches']

11443     [(Isaac Pierce, 91.66666666666666, 12720), (Is...
11444                                                    []
11445                                                    []
11446                                                    []
11447                                                    []
                                ...                        
119754                                                   []
119755                                                   []
119756                                                   []
119757                                                   []
119758                                                   []
Name: matches, Length: 27862, dtype: object

In [1]:
agg_debt_ma_parsed = agg_debt_ma.loc[agg_debt_ma.matches.str.len() > 0]

NameError: name 'agg_debt_ma' is not defined

In [36]:
agg_debt_ma_parsed.to_csv('data/similar_names_ma.csv')